<a href="https://colab.research.google.com/github/marconoriega0703-sys/MATH_120/blob/main/Week9_Asynchronous_Text_Analysis_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Week 9 Asynchronous Assignment — From Data Structures to Text Analysis & Generation

**Deliverable:** Submit this completed notebook on GitHub (`assignments/week9/`).

---

## 🧭 Before You Begin: Background Resources (15–25 min total)

### 📘 Readings

* **“N-grams and Markov Chains” (decontextualize)** — https://www.decontextualize.com/teaching/rwet/n-grams-and-markov-chains/  

### 🎥 Short videos
1. Introducing Markov Chains - https://youtu.be/JHwyHIz6a8A?si=mlbkGrhewP2eJJoT

2. Introduction to N-Grams - https://www.youtube.com/watch?v=hM49MPmakNI


### 🎧 Optional podcast context
- Radiolab - [Talking to Machines](https://open.spotify.com/episode/2XCSL3ntSjclhBQPAFg6oS?si=2c8bdd2f07924eaa)


## Learning objectives
- Use **lists**, **tuples**, and **dictionaries** to organize and analyze text.
- Build a **token frequency table** and **bigram / n-gram** index.
- Implement a simple **Markov (n-gram) text generator** using tuple contexts.
- Reflect on **limitations and ethics** of text generation.



## Setup

Run the cell below to set up helper functions and a small sample corpus.  
You may replace `CORPUS_TEXT` with your own **original writing** (short paragraph).


In [2]:
# --- SETUP ---
import re, random
from collections import Counter

# A small sample corpus; replace with your own short text if you prefer.
CORPUS_TEXT = (
    "all children grow up, except one. they soon know that they will grow up, "
    "and the way they know it is that they begin to be like other people. "
    "it is then that, if you are careful, you can see it happening."
)

def normalize(text):
    # Lowercase, separate punctuation, collapse whitespace, split on spaces
    text = text.lower()
    text = re.sub(r'([.,;:!?()])', r' \1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.split(' ')

def make_unigram_counts(tokens):
    return Counter(tokens)

def make_next_tokens(tokens):
    d = {}
    for i in range(len(tokens)-1):
        cur, nxt = tokens[i], tokens[i+1]
        d.setdefault(cur, []).append(nxt)
    return d

def make_ngram_index(tokens, n=2):
    # Map tuple context of length n -> list of next tokens
    index = {}
    for i in range(len(tokens)-n):
        ctx = tuple(tokens[i:i+n])
        nxt = tokens[i+n]
        index.setdefault(ctx, []).append(nxt)
    return index

def choose_weighted(counter_like):
    # counter_like can be an iterable of tokens or a dict mapping token->count
    c = Counter(counter_like)
    items, weights = zip(*c.items())
    return random.choices(items, weights=weights, k=1)[0]

def generate_markov(index, seed, length=40):
    n = len(seed)
    out = list(seed)
    while len(out) < length:
        ctx = tuple(out[-n:])
        options = index.get(ctx)
        if not options:
            break
        out.append(choose_weighted(options))
    return " ".join(out)

tokens = normalize(CORPUS_TEXT)
print("Preview tokens:", tokens[:15], "...")


Preview tokens: ['all', 'children', 'grow', 'up', ',', 'except', 'one', '.', 'they', 'soon', 'know', 'that', 'they', 'will', 'grow'] ...



## Part 1 — Quick review (tuples, lists, dictionaries)  

**Your turn 1.1**  
- Create `seed_bigram`: a **tuple** with the first 2 tokens.  
- Create `rest`: a **list** of the remaining tokens.  
- Create `pos_map`: a **dict** mapping each unique token → its **first** index in `tokens`.  
- Briefly explain (in a comment) why a **tuple** (immutable) is a good choice for n-gram contexts.


In [74]:
# TODO: Your turn 1.1
seed_bigram = tuple(tokens[:2])
rest = list[tokens[2:]]
pos_map = {token: tokens.index(token) for token in set(tokens)}
# Tuples are preferred over lists and dicts to represent n-grams as tuples can be used as a dictionary key.
# A dictionary key must be immutable and tuples are immutable, while lists and dicts are not.

Tuples are preferred over lists and dicts to represent n-grams as tuples can be used as a dictionary key. A dictionary key must be immutable and tuples are immutable, while lists and dicts are not.


## Part 2 — Frequency analysis

**Your turn 2.1**  
- Build `freq` = dict/Counter of **token → count**.  
- Show the **top 10** tokens.


In [78]:
# TODO: Your turn 2.1
freq = Counter(tokens) # freq counts the number of times a token appears.
for token, count in freq.most_common(10): # most_common(n) returns the n most common tokens in the corpus.
    print(f"{token}: {count}")

,: 4
they: 4
.: 3
that: 3
it: 3
grow: 2
up: 2
know: 2
is: 2
you: 2



## Part 3 — Bigram next-token index
**Your turn 3.1**  
- Build `next_tokens` mapping token → list of tokens that **follow** it somewhere.  
- Print the first 5 entries in the form `token -> [sample next tokens]`.


In [83]:
# TODO: Your turn 3.1
next_tokens = make_next_tokens(tokens)
for token, next_token in list(next_tokens.items())[:5]: # The entry in [] is the next token whenever the word appears in the corpus.
    print(f"{token} -> {next_token}") # This is why grow has two entries of "up" in [] as grow appears twice in the text with "up" being the next token both times.

all -> ['children']
children -> ['grow']
grow -> ['up', 'up']
up -> [',', ',']
, -> ['except', 'and', 'if', 'you']



## Part 4 — General n-grams with tuple keys

**Your turn 4.1**  
- Use `make_ngram_index(tokens, n=2)` and `n=3`.  
- Inspect a few keys and values; explain how tuples serve as fixed-length contexts.


In [98]:
# TODO: Your turn 4.1
ngram_index = make_ngram_index(tokens, n=2)
for key, value in tuple(ngram_index.items())[:5]: # The entry in [] is the next token that is after two specific tokens.
    print(f"{key} -> {value}") # Ex: "," appears after "up" twice with "except" being the next token in the first instance and "and" being the next token in the second.

('all', 'children') -> ['grow']
('children', 'grow') -> ['up']
('grow', 'up') -> [',', ',']
('up', ',') -> ['except', 'and']
(',', 'except') -> ['one']


In [101]:
ngram_index = make_ngram_index(tokens, n=3)
for key, value in tuple(ngram_index.items())[:5]:
    print(f"{key} -> {value}")

('all', 'children', 'grow') -> ['up']
('children', 'grow', 'up') -> [',']
('grow', 'up', ',') -> ['except', 'and']
('up', ',', 'except') -> ['one']
(',', 'except', 'one') -> ['.']


Tuples serve as fixed-length contexts as they are immutable. Because of this, the length of the key will always be the value you assign to n.


## Part 5 — Tiny Markov text generator

**Your turn 5.1**  
- Use the **bigram** model first (`n=2`).  
- Pick a `seed` (tuple of length 2) from your tokens.  
- Generate 30–50 tokens.  
- Try again with `n=3` and compare.


In [143]:
# TODO: Your turn 5.1
bigram_index = make_ngram_index(tokens, n=2)
seed = tuple(tokens[:2])
print(generate_markov(bigram_index, seed, length=30))

all children grow up , and the way they know it is then that , if you are careful , you can see it happening .


In [144]:
bigram_index = make_ngram_index(tokens, n=2)
seed = tuple(tokens[:2])
print(generate_markov(bigram_index, seed, length=40))

all children grow up , except one . they soon know that they will grow up , and the way they know it is that they will grow up , and the way they know it is that they will


In [145]:
bigram_index = make_ngram_index(tokens, n=3)
seed = tuple(tokens[:3])
print(generate_markov(bigram_index, seed, length=30))

all children grow up , except one . they soon know that they will grow up , and the way they know it is that they begin to be like


In [146]:
bigram_index = make_ngram_index(tokens, n=3)
seed = tuple(tokens[:3])
print(generate_markov(bigram_index, seed, length=40))

all children grow up , and the way they know it is that they begin to be like other people . it is then that , if you are careful , you can see it happening .


The Markov Model with the order of 3 has more comprehensible text than the model that has an order of 2 as the model with the higher order considers more tokens meaning that it's more likely to idenify patterns of the text.


## Reflection

- One **advantage** and one **limitation** of an n-gram Markov model.  

In [147]:
reflection = """
An advantage of a Markov model is that the model will always return an output as the model will move to the next iteration if current iteration can't find the n-gram.
A limitation of a Markov model is that the model is not reliable for generating longer text as it's predictions are based on small sequences of tokens, not the entire context of said text.
"""